In [6]:
import numpy as np
import pandas as pd
import json

# For each region, create a list of its GID

## Load the IVS data grouped by region

In [7]:
IVS_region = pd.read_csv("out/IVS_grouped_by_country_region.csv", encoding="utf-16", low_memory=False)
IVS_region.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,EVS,1981-1984,56,Belgium,BEL,56021,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,EVS,1981-1984,56,Belgium,BEL,56022,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EVS,1981-1984,56,Belgium,BEL,56023,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,EVS,1981-1984,56,Belgium,BEL,56024,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EVS,1981-1984,56,Belgium,BEL,56025,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Load the region codes to GID dictionaries and add the list of GID to the IVS data

## WVS/EVS

In [8]:
VWS_to_gid = pd.read_csv("gid/region_codes_to_GID_WVS_EVS.csv", low_memory=False)
VWS_to_gid.drop("Region Name", inplace=True, axis=1)
#remove empty lines
VWS_to_gid.dropna(how="all", inplace=True) 
VWS_to_gid['Country Numeric'] = VWS_to_gid['Country Numeric'].astype(np.int64)
VWS_to_gid['Region Numeric'] = VWS_to_gid['Region Numeric'].astype(np.int64)
VWS_to_gid

,Survey,Wave,Country Numeric,Country ISO3,Region Numeric,gid
0,WVS,2010-2014,840,USA,840001,USA.20_1
1,WVS,2010-2014,840,USA,840001,USA.46_1
2,WVS,2010-2014,840,USA,840001,USA.30_1
3,WVS,2010-2014,840,USA,840001,USA.22_1
4,WVS,2010-2014,840,USA,840001,USA.40_1
...,...,...,...,...,...,...
818,WVS,2010-2014,752,SWE,752020,SWE.17_1
819,WVS,2010-2014,752,SWE,752026,SWE.18_1
820,WVS,2010-2014,752,SWE,752024,SWE.19_1
821,WVS,2010-2014,752,SWE,752021,SWE.20_1


In [9]:
VWS_to_list_gid = VWS_to_gid.groupby(list(VWS_to_gid.columns[:-1])).agg({'gid':lambda x: ':'.join(list(x))}).reset_index()
VWS_to_list_gid

,Survey,Wave,Country Numeric,Country ISO3,Region Numeric,gid
0,WVS,1999-2004,50,BGD,50001,BGD.3.1_1
1,WVS,1999-2004,50,BGD,50002,BGD.2.4_1
2,WVS,1999-2004,50,BGD,50003,BGD.1.2_1
3,WVS,1999-2004,50,BGD,50004,BGD.3.10_1
4,WVS,1999-2004,50,BGD,50005,BGD.3.17_1
...,...,...,...,...,...,...
367,WVS,2010-2014,840,USA,840005,USA.4_1:USA.19_1:USA.37_1:USA.44_1
368,WVS,2010-2014,840,USA,840006,USA.14_1:USA.15_1:USA.23_1:USA.36_1:USA.50_1
369,WVS,2010-2014,840,USA,840007,USA.16_1:USA.17_1:USA.24_1:USA.26_1:USA.28_1:U...
370,WVS,2010-2014,840,USA,840008,USA.3_1:USA.6_1:USA.13_1:USA.27_1:USA.29_1:USA...


In [10]:
len(VWS_to_list_gid["Region Numeric"].unique())

372

In [11]:
IVS_subset_VWS = pd.merge(IVS_region, VWS_to_list_gid, 
                on = ['Survey', 'Wave', 'Country Numeric', 'Country ISO3',
                                    'Region Numeric'],
                sort=False)

In [12]:
IVS_subset_VWS.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10,gid
0,WVS,1999-2004,50,Bangladesh,BGD,50001,BD: Dhaka,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BGD.3.1_1
1,WVS,1999-2004,50,Bangladesh,BGD,50002,BD: Chittagong,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BGD.2.4_1
2,WVS,1999-2004,50,Bangladesh,BGD,50003,BD: Barisal,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BGD.1.2_1
3,WVS,1999-2004,50,Bangladesh,BGD,50004,BD: Mymensingh,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BGD.3.10_1
4,WVS,1999-2004,50,Bangladesh,BGD,50005,BD: Tangail,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BGD.3.17_1


In [13]:
# lets check if we left something out
VWS_to_list_gid[~VWS_to_list_gid["Region Numeric"].isin(IVS_subset_VWS["Region Numeric"])]

,Survey,Wave,Country Numeric,Country ISO3,Region Numeric,gid


## NUTS

In [14]:
NUTS_to_gid = pd.read_csv("gid/region_codes_to_GID_NUTS.csv", low_memory=False)
NUTS_to_gid.drop(["Region Name", "Region Numeric", "NUTS3"], inplace=True, axis=1)
#remove empty lines
NUTS_to_gid.dropna(how="all", inplace=True) 
NUTS_to_gid['Country Numeric'] = NUTS_to_gid['Country Numeric'].astype(np.int64)

NUTS_to_gid

,Survey,Wave,Country Numeric,Country ISO3,NUTS1,NUTS2,gid
0,EVS,2008-2010,372,IRL,IE0,-1,IRL
1,EVS,2008-2010,372,IRL,IE0,IE01,IRL.16_1
2,EVS,2008-2010,372,IRL,IE0,IE01,IRL.20_1
3,EVS,2008-2010,372,IRL,IE0,IE01,IRL.15_1
4,EVS,2008-2010,372,IRL,IE0,IE01,IRL.11_1
...,...,...,...,...,...,...,...
401,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM6,GBR.3.17_1
402,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM6,GBR.3.20_1
403,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM6,GBR.3.21_1
404,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM6,GBR.3.23_1


In [15]:
len(NUTS_to_gid.NUTS2.unique())

103

In [16]:
NUTS_to_list_gid = NUTS_to_gid.groupby(list(NUTS_to_gid.columns[:-1])).agg({'gid':lambda x: ':'.join(list(x))}).reset_index()
NUTS_to_list_gid

,Survey,Wave,Country Numeric,Country ISO3,NUTS1,NUTS2,gid
0,EVS,2008-2010,56,BEL,BE1,BE10,BEL.1.1_1
1,EVS,2008-2010,56,BEL,BE2,BE21,BEL.2.1_1
2,EVS,2008-2010,56,BEL,BE2,BE22,BEL.2.2_1
3,EVS,2008-2010,56,BEL,BE2,BE23,BEL.2.3_1
4,EVS,2008-2010,56,BEL,BE2,BE24,BEL.2.4_1
...,...,...,...,...,...,...,...
98,EVS,2008-2010,826,GBR,GB-GBNL,GB-GBNL2,GBR.4.13_1:GBR.4.15_1:GBR.4.5_1:GBR.4.21_1:GBR...
99,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM2,GBR.3.3_1:GBR.3.5_1:GBR.3.7_1:GBR.3.10_1:GBR.3...
100,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM3,GBR.3.4_1:GBR.3.6_1:GBR.3.8_1:GBR.3.9_1:GBR.3....
101,EVS,2008-2010,826,GBR,GB-GBNM,GB-GBNM6,GBR.3.4_1:GBR.3.13_1:GBR.3.17_1:GBR.3.20_1:GBR...


In [17]:
IVS_subset_NUTS = pd.merge(IVS_region, NUTS_to_list_gid, 
                on = ['Survey', 'Wave', 'Country Numeric', 'Country ISO3',
                      'NUTS1', 'NUTS2'],
                sort=False)

In [18]:
IVS_subset_NUTS

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10,gid
0,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE1,BE10,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BEL.1.1_1
1,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE21,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BEL.2.1_1
2,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE22,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BEL.2.2_1
3,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE23,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BEL.2.3_1
4,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE24,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BEL.2.4_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNL,GB-GBNL2,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GBR.4.13_1:GBR.4.15_1:GBR.4.5_1:GBR.4.21_1:GBR...
99,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM2,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GBR.3.3_1:GBR.3.5_1:GBR.3.7_1:GBR.3.10_1:GBR.3...
100,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM3,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GBR.3.4_1:GBR.3.6_1:GBR.3.8_1:GBR.3.9_1:GBR.3....
101,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM6,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GBR.3.4_1:GBR.3.13_1:GBR.3.17_1:GBR.3.20_1:GBR...


## Concatenate the two dataframes

In [19]:
IVS_region = pd.concat([IVS_subset_VWS, IVS_subset_NUTS], ignore_index=True)
IVS_region

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10,gid
0,WVS,1999-2004,50,Bangladesh,BGD,50001,BD: Dhaka,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BGD.3.1_1
1,WVS,1999-2004,50,Bangladesh,BGD,50002,BD: Chittagong,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BGD.2.4_1
2,WVS,1999-2004,50,Bangladesh,BGD,50003,BD: Barisal,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BGD.1.2_1
3,WVS,1999-2004,50,Bangladesh,BGD,50004,BD: Mymensingh,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BGD.3.10_1
4,WVS,1999-2004,50,Bangladesh,BGD,50005,BD: Tangail,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BGD.3.17_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNL,GB-GBNL2,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GBR.4.13_1:GBR.4.15_1:GBR.4.5_1:GBR.4.21_1:GBR...
471,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM2,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GBR.3.3_1:GBR.3.5_1:GBR.3.7_1:GBR.3.10_1:GBR.3...
472,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM3,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GBR.3.4_1:GBR.3.6_1:GBR.3.8_1:GBR.3.9_1:GBR.3....
473,EVS,2008-2010,826,United Kingdom of Great Britain and Northern I...,GBR,-5,Unknown,GB-GBNM,GB-GBNM6,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GBR.3.4_1:GBR.3.13_1:GBR.3.17_1:GBR.3.20_1:GBR...


# Create dictionary with the anwers descriptions

In [22]:
IVS_variables = pd.read_csv("input/IVS_Variable_List.csv")

In [23]:
IVS_variables

,Included,Name,Label,Last EVS,Last WVS,Categories
0,1,A001,Important in life: Family,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
1,1,A002,Important in life: Friends,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
2,0,A003,Important in life: Leisure time,EVS 2008,WVS6 (2010-2012),-5:Missing; Unknown\n-4:Not asked in survey\n-...
3,0,A004,Important in life: Politics,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
4,1,A005,Important in life: Work,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
...,...,...,...,...,...,...
1422,0,w007,Spouse/partner had/has how many employees,EVS 2008,NaN,1:None\n2:1-9\n3:10-24\n4:25 or more\n-5:Missi...
1423,0,w008,Does spouse/partner supervise someone,EVS 2008,NaN,0:No\n1:Yes\n-5:Missing; Unknown\n-4:Not asked...
1424,0,w009,How many people does she/he supervise,EVS 2008,NaN,1:1-9\n2:10-24\n3:25 or more\n-5:Missing; Unkn...
1425,0,w010,Spouse/partner experienced unemployment longer...,EVS 2008,NaN,0:No\n1:Yes\n-5:Missing; Unknown\n-4:Not asked...


In [24]:
IVS_variables = IVS_variables.loc[IVS_variables.Included == 1].copy()
IVS_variables.drop(["Included", "Last EVS", "Last WVS"], axis=1, inplace=True)

In [25]:
cat_dict_column = []

for idx, row in IVS_variables.iterrows():
    codes = {}
    for line in row.Categories.split("\n")[:-1]: # there is always a \n at the end
        try:
            key = line.split(":")[0]
            value = line.split(":")[1]
            codes[key] = value
        except:
            continue
    cat_dict_column.append(codes)

In [26]:
IVS_variables["Categories"] = cat_dict_column

In [27]:
IVS_variables.set_index("Name", inplace=True)

In [28]:
variables_dict = IVS_variables.to_dict(orient="index")

## We need to pop some keys
ZMEN_010 is not present in the dataset

the following ones are the values we aggregated on

S002
S002EVS
S001
S017

X048WVS
X048
x048a_n1
x048b_n2
x048c_n3

In [29]:
len(variables_dict.keys())

87

In [30]:
to_drop = ["ZMEN_010", "S002", "S002EVS", "S001", "S017",
          "X048WVS", "X048", "x048a_n1", "x048b_n2", "x048c_n3"]


for key in to_drop:
    variables_dict.pop(key)

len(variables_dict.keys())

77

# Dump the answers in a json

In [31]:
IVS_region['json'] = IVS_region.apply(lambda x: x.to_dict(), axis=1)

In [32]:
# now we have a flat dict, we will need to organize it
IVS_region.json[0]

{'Survey': 'WVS',
 'Wave': '1999-2004',
 'Country Numeric': 50,
 'Country Name': 'Bangladesh',
 'Country ISO3': 'BGD',
 'Region Numeric': 50001,
 'Region Name': 'BD: Dhaka',
 'NUTS1': ' ',
 'NUTS2': ' ',
 'NUTS3': ' ',
 'Sample Size': 199,
 'A001_-5': 0.0,
 'A001_-4': 0.0,
 'A001_-2': 0.0,
 'A001_-1': 0.0,
 'A001_1': 0.9547738693467336,
 'A001_2': 0.04522613065326633,
 'A001_3': 0.0,
 'A001_4': 0.0,
 'A002_-5': 0.0,
 'A002_-4': 0.0,
 'A002_-2': 0.0,
 'A002_-1': 0.005025125628140704,
 'A002_1': 0.3165829145728643,
 'A002_2': 0.5376884422110553,
 'A002_3': 0.12562814070351758,
 'A002_4': 0.01507537688442211,
 'A005_-5': 0.0,
 'A005_-4': 0.0,
 'A005_-3': 0.0,
 'A005_-2': 0.0,
 'A005_-1': 0.01507537688442211,
 'A005_1': 0.8542713567839196,
 'A005_2': 0.12060301507537688,
 'A005_3': 0.010050251256281407,
 'A005_4': 0.0,
 'A009_-5': 0.0,
 'A009_-4': 0.0,
 'A009_-2': 0.0,
 'A009_-1': 0.005025125628140704,
 'A009_1': 0.20603015075376885,
 'A009_2': 0.4974874371859296,
 'A009_3': 0.246231155778

In [33]:
properties = []

for idx, row in IVS_region.iterrows():
    print(idx)
    new_dict = {}
    
    # for each row we create a dict
    # {..., 
    #  column_id: {label: 'description',
    #              frequencies: {dict with the anwers onehot encoded}
    #.             categories: {dict with the answers codes}}, 
    #  ...}
    
    # label and categories are from variables_dict
    
    # add the original region labels
    new_dict["original region code"]= {
        'Region Numeric': row.json['Region Numeric'],
        'Region Name': row.json['Region Name'],
        'NUTS1': row.json['NUTS1'],
        'NUTS2': row.json['NUTS2'],
        'NUTS3': row.json['NUTS3'],
    }
    
    
    for answer_code in variables_dict.keys():
        
        # we already have label and categories
        sub_dict = variables_dict[answer_code]
        
        # we need frequencies
        # in the row json we have a flat dict
        # we want to aggregate by name of the answer
        sub_dict["Frequencies"] = {}
        
        for key, item in row.json.items():
            if key.startswith(answer_code):
                sub_dict["Frequencies"][key] = item
        new_dict[answer_code] = sub_dict
        
    properties.append(new_dict)    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [49]:
df_to_save = IVS_region[ ['Survey', 'Wave', 'Country Numeric', 
                           'Country Name', 'Country ISO3', 'Sample Size', 'gid',
                           'Region Numeric','Region Name', 'NUTS1', 'NUTS2', 'NUTS3'] ].copy()
df_to_save["Properties"] = properties

In [50]:
df_to_save.to_csv("to_upload/IVS_region_GID.csv", index=False)

# UPLOAD
## Loads the data from csv, there is no need to run the cells above

In [37]:
import pandas
import psycopg2

In [52]:
def create_IVS_region_table(cur):
    sql_create_country_stats_table = """ 
        CREATE TABLE IF NOT EXISTS IVS_region (
            survey text NOT NULL,
            wave text NOT NULL,
            country text NOT NULL,
            countrycode text NOT NULL,
            region text,
            regioncode text,
            NUTS1 text,
            NUTS2 text,
            NUTS3 text,
            sample_size integer NOT NULL,
            gid text NOT NULL,
            properties jsonb,
            UNIQUE (survey, wave, country, countrycode, gid),
            PRIMARY KEY (survey, wave, country, countrycode, gid)
        )"""

    cur.execute(sql_create_country_stats_table)

    print("IVS_country table created")

In [39]:
def drop_IVS_region_table(cur):
    sql_drop_IVS_region_table = """DROP TABLE IVS_region"""
    
    try:
        cur.execute(sql_drop_IVS_region_table)
        conn.commit() 
        print("IVS_region table deleted")
    except:
        print("IVS_region not present")

In [55]:
def import_IVS_region_data(cur,PATH):
    
    INSERT_STATEMENT = 'INSERT INTO IVS_region \
                            (survey, wave, \
                             country, countrycode,  \
                             region, regioncode, \
                             NUTS1, NUTS2, NUTS3, \
                             sample_size, gid, \
                             properties) \
                        VALUES (%s, %s, \
                                %s, %s, \
                                %s, %s, \
                                %s, %s, %s, \
                                %s, %s, \
                                %s);'
    
    data = pd.read_csv(PATH)

    for idx, row in data.iterrows():
        print(idx)
        cur.execute(INSERT_STATEMENT, (row['Survey'], row['Wave'], 
                                       row['Country Name'], row['Country ISO3'], 
                                       row['Region Name'], row['Region Numeric'],
                                       row['NUTS1'], row['NUTS2'], row['NUTS3'],
                                       row['Sample Size'], row['gid'],
                                       json.dumps(row["Properties"])))

In [56]:
# Connect to covid19db.org
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19db_adm_play',
    user='covid19db_adm_rw',
    password='fGt962FdeG2yXj3c4d3'
)

cur = conn.cursor()

drop_IVS_region_table(cur)
conn.commit()


create_IVS_region_table(cur)
conn.commit()

# there are 475 rows 
import_IVS_region_data(cur, "to_upload/IVS_region_GID.csv")
conn.commit()


IVS_region table deleted
IVS_country table created
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264

In [57]:
cur.execute("""SELECT * FROM IVS_region""")

data = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
data


,survey,wave,country,countrycode,region,regioncode,nuts1,nuts2,nuts3,sample_size,gid,properties
0,WVS,1999-2004,Bangladesh,BGD,BD: Dhaka,50001,,,,199,BGD.3.1_1,{'original region code': {'Region Numeric': 50...
1,WVS,1999-2004,Bangladesh,BGD,BD: Chittagong,50002,,,,121,BGD.2.4_1,{'original region code': {'Region Numeric': 50...
2,WVS,1999-2004,Bangladesh,BGD,BD: Barisal,50003,,,,105,BGD.1.2_1,{'original region code': {'Region Numeric': 50...
3,WVS,1999-2004,Bangladesh,BGD,BD: Mymensingh,50004,,,,128,BGD.3.10_1,{'original region code': {'Region Numeric': 50...
4,WVS,1999-2004,Bangladesh,BGD,BD: Tangail,50005,,,,81,BGD.3.17_1,{'original region code': {'Region Numeric': 50...
...,...,...,...,...,...,...,...,...,...,...,...,...
470,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,Unknown,-5,GB-GBNL,GB-GBNL2,-5,22,GBR.4.13_1:GBR.4.15_1:GBR.4.5_1:GBR.4.21_1:GBR...,{'original region code': {'Region Numeric': -5...
471,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,Unknown,-5,GB-GBNM,GB-GBNM2,-5,43,GBR.3.3_1:GBR.3.5_1:GBR.3.7_1:GBR.3.10_1:GBR.3...,{'original region code': {'Region Numeric': -5...
472,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,Unknown,-5,GB-GBNM,GB-GBNM3,-5,43,GBR.3.4_1:GBR.3.6_1:GBR.3.8_1:GBR.3.9_1:GBR.3....,{'original region code': {'Region Numeric': -5...
473,EVS,2008-2010,United Kingdom of Great Britain and Northern I...,GBR,Unknown,-5,GB-GBNM,GB-GBNM6,-5,6,GBR.3.4_1:GBR.3.13_1:GBR.3.17_1:GBR.3.20_1:GBR...,{'original region code': {'Region Numeric': -5...
